EDM Competition 2019-2020

Objective: Scrape csv training data for meaningful insights
Write CSV containing model grade data

In [ ]:
import csv,datetime
import pandas as pd

import math


#Scrape 1:
# Time Spent on each Accession
# Number of times each observable occurs
# Total Time spent on question types
# Number of questions completed
# Number of total actions
# Number of times revisiting a question
############################################
def scrape1(trainData, outCSV, labelData=None, hidden=False):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]

    startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    actionCount = 0
    curData = {'StudentID' : curStudent['STUDENTID']}
    if(not hidden):
        labelStudent = labelData.iloc[0,:]
        curData['Target'] = labelStudent.loc['EfficientlyCompletedBlockB']
    entryList = []
    revisits = 0

    questionCount = 0
    studentCount = 0
    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 100000 == 0):
            print(rowCount)
        actionCount +=1


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            curData['NumActs'] = actionCount
            curData['NumQuests'] = questionCount
            curData['Revisits'] = revisits
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): 
                curStudent = row[1]
                curData = {'StudentID' : curStudent['STUDENTID']}
                if(not hidden):
                    labelStudent = labelData.iloc[studentCount,:]
                    curData['Target'] = labelStudent.loc['EfficientlyCompletedBlockB']

            startTime=endTime=qStart=qEnd = parseTime(row[1].loc['EventTime'])
            actionCount=0
            revisits = 0
            questionCount=0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):

            #if we have NOT seen this question before
            if(row[1].loc['AccessionNumber'] not in curData.keys()):
                questionCount +=1
                curData[row[1].loc['AccessionNumber']] = (qEnd-qStart).total_seconds()
            else:
                revisits +=1
                curData[row[1].loc['AccessionNumber']] += (qEnd-qStart).total_seconds()

            if(row[1].loc['ItemType'] not in curData.keys()):
                curData[row[1].loc['ItemType']] = (qEnd - qStart).total_seconds()
            else:
                curData[row[1].loc['ItemType']] += (qEnd-qStart).total_seconds()

            #update the starting time for a new question
            if(str(row[1].loc['EventTime']) != 'nan'): qStart=qEnd=parseTime(row[1].loc['EventTime'])

         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=parseTime(row[1].loc['EventTime'])

        #Add new observable
        if(row[1].loc['Observable'] not in curData.keys()): curData[row[1].loc['Observable']] =1

        #Add to the count of this observable
        else: curData[row[1].loc['Observable']] +=1

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)
    
#Scrape2
#Process Scrape
# For each process:
#    Number of times the process occurred
#    Total time spent on this process
#    Average time spent on this process
############################################
def scrape2(trainData, procList, outCSV):
    dataList = []
    for row in trainData.itertuples():
        #break between students
        dataList.append([row.STUDENTID, row.Observable,row.EventTime])

    curID = dataList[0][0]
    count = 0
    curActions = []
    studentList =[]
    finalList = []
    for i in dataList:
        #add all actions for current student to curActions
        if(curID == i[0]):
            curActions.append(i)
        #break between students
        else:
            studentList.append([curID,curActions])
            curActions = []
            curID = i[0]
    studentList.append([curID,curActions])

    i = j = 0
    while(i<len(studentList)):
        commonProcs = {}
        #print(i)
        j=0
        while(j<len(procList)):
            commonProcs = compareProcs2(commonProcs,studentList[i][1],procList[j])
            j+=1
        commonProcs['StudentID'] = studentList[i][0]
        finalList.append(commonProcs)
        i+=1
    outData = pd.DataFrame(finalList)
    outData.to_csv(outCSV,index=False)
    

#scrape3
# Starting Time
# Time spent on each observable
# ActionRate - amount of time taken every 10 clicks
############################################
def scrape3(trainData, outCSV, labelData=None):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    #labelStudent = labelData.iloc[0,:]
    actionTimerEnd=actionTimer=startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    #nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID'], 'StartTime':parseTime(curStudent['EventTime']).total_seconds()}
    entryList = []

    questionCount = 0
    studentCount = 0
    rowCount= 0
    priorObs = startTime

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 100000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=endTime=qStart=qEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}
            curData['StartTime'] = parseTime(row[1].loc['EventTime']).total_seconds()
            actionCount=0
            priorObs = startTime
            questionCount=0
            reviewTime = 0

        #break between questions
        elif(row[1].loc['AccessionNumber'] != trainData.iloc[row[0]-1,:].loc['AccessionNumber']):

            #update the starting time for a new question
            if(str(row[1].loc['EventTime']) != 'nan'): qStart=qEnd=parseTime(row[1].loc['EventTime'])

         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=actionTimerEnd=obsTimer=parseTime(row[1].loc['EventTime'])

        #Add new observable
        if(row[1].loc['Observable']+"_T" not in curData.keys()):
            curData[row[1].loc['Observable']+"_T"] = (obsTimer-priorObs).total_seconds()

        #Add to the count of this observable
        else:
            curData[row[1].loc['Observable']+"_T"] += (obsTimer-priorObs).total_seconds()

        #add time for last 10 actions
        if(actionCount%10 == 0 and actionCount<600):
            curData["actionRate" +str(actionCount)] = (actionTimerEnd-actionTimer).total_seconds()
            actionTimer=actionTimerEnd=parseTime(row[1].loc['EventTime'])

        priorObs =obsTimer

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)

#scrape4
# Click Rate - Number of clicks taken per minute every minute
############################################
def scrape4(trainData, outCSV):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    #labelStudent = labelData.iloc[0,:]
    actionTimerEnd=actionTimer=startTime=endTime=qStart=qEnd = parseTime(curStudent['EventTime'])
    #nonQuestions = ['Directions','BlockReview','TimeLeftMessage','Help']
    actionCount = 0
    actionMark = 0
    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID']}
    entryList = []

    questionCount = 0
    studentCount = 0
    rowCount= 0
    priorObs = startTime

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 100000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)
            studentCount+=1

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=actionTimerEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}
            actionCount=0
            actionMark = 0


         #update the ending time with each row
        if(str(row[1].loc['EventTime']) != 'nan'): endTime=qEnd=actionTimerEnd=obsTimer=parseTime(row[1].loc['EventTime'])

        #add time for last 10 actions
        if((actionTimerEnd - actionTimer).total_seconds()>60):
            curData["clickRate" +str(actionMark)] = actionCount
            actionTimer=actionTimerEnd=parseTime(row[1].loc['EventTime'])
            actionMark +=1
            actionCount = 0

        priorObs =obsTimer

    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)

#scrape5
# Clicks per accession type
# Clicks per question
############################################
def scrape5(trainData, outCSV):
    #initialize all tracking variables
    curStudent = trainData.iloc[0,:]
    actionCount = 0

    obsTimer = 0
    reviewTime = 0
    curData = {'StudentID' : curStudent['STUDENTID']}
    entryList = []

    rowCount= 0

    for row in trainData.iterrows():
        rowCount +=1
        if(rowCount % 100000 == 0):
            print(rowCount)
        actionCount +=1

        observed = row[1].loc['Observable']


        #print(rowCount)
        #break between students - Write to row
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            entryList.append(curData)

            #update tracking vars for next student if not last student
            if(row[0] != len(trainData)-1): curStudent = row[1]
            actionTimer=startTime=actionTimerEnd = parseTime(row[1].loc['EventTime'])

            curData = {'StudentID' : curStudent['STUDENTID']}

        # Add new ItemType
        if (row[1].loc['ItemType']+"ObsClicks" not in curData.keys()):
            curData[row[1].loc['ItemType']+"ObsClicks"] = 1

            # Add to the count of this observable
        else:
            curData[row[1].loc['ItemType']+"ObsClicks"] += 1

        # Add new Question
        if (row[1].loc['AccessionNumber']+"QClicks" not in curData.keys()):
            curData[row[1].loc['AccessionNumber']+"QClicks"] = 1

            # Add to the count of this observable
        else:
            curData[row[1].loc['AccessionNumber']+"QClicks"] += 1


    outData = pd.DataFrame(entryList)
    outData.to_csv(outCSV,index=False)
    
#addStats

def addStats(featureData, outCSV):
    data = pd.read_csv(featureData)
    
    #stats for average process times
    pt_r_Col = [col for col in data if (col.endswith('_PT_R'))]
    pt_r_DF = data[pt_r_Col]

    rowList = []

    for row in pt_r_DF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['mean_PT_R'] = description['mean']
        rowDict['sd_PT_R'] = description['std']
        rowDict['max_PT_R'] = description['max']
        rowDict['min_PT_R'] = description['min']
        rowDict['range_PT_R'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)
    
    #stats for total process times
    pt_Col = [col for col in data if (col.endswith('_PT'))]
    pt_DF = data[pt_Col]
    
    rowList = []

    for row in pt_DF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['mean_PT'] = description['mean']
        rowDict['sd_PT'] = description['std']
        rowDict['max_PT'] = description['max']
        rowDict['min_PT'] = description['min']
        rowDict['range_PT'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)
    
    #stats for action rate
    
    actionCol = [col for col in data if col.startswith('actionRate')]
    actionDF = data[actionCol]

    rowList = []

    for row in actionDF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['meanAction'] = description['mean']
        rowDict['sdAction'] = description['std']
        rowDict['maxAction'] = description['max']
        rowDict['minAction'] = description['min']
        rowDict['rangeAction'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)
    
    #stats for click rate

    cr_Col = [col for col in data if col.startswith('clickRate')]
    cr_DF = data[cr_Col]

    rowList = []

    for row in cr_DF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['meanCR'] = description['mean']
        rowDict['sdCR'] = description['std']
        rowDict['maxCR'] = description['max']
        rowDict['minCR'] = description['min']
        rowDict['rangeCR'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)

    #stats for question times
    questionCol = [col for col in data if (col.startswith('VH') and not col.endswith('Clicks'))]
    questionsDF = data[questionCol]
    questionsDF.head()

    rowList = []

    for row in questionsDF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['meanQ'] = description['mean']
        rowDict['sdQ'] = description['std']
        rowDict['maxQ'] = description['max']
        rowDict['minQ'] = description['min']
        rowDict['rangeQ'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)

    #stats for question clicks
    questionCol = [col for col in data if (col.startswith('VH') and col.endswith('Clicks'))]
    questionsDF = data[questionCol]
    questionsDF.head()

    rowList = []

    for row in questionsDF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['meanQClicks'] = description['mean']
        rowDict['sdQClicks'] = description['std']
        rowDict['maxQClicks'] = description['max']
        rowDict['minQClicks'] = description['min']
        rowDict['rangeQClicks'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)

    #stats for observation clicks
    questionCol = [col for col in data if col.endswith('ObsClicks')]
    questionsDF = data[questionCol]
    questionsDF.head()

    rowList = []

    for row in questionsDF.iterrows():
        rowDict = {}
        description = row[1].describe()
        rowDict['meanObsClicks'] = description['mean']
        rowDict['sdObsClicks'] = description['std']
        rowDict['maxObsClicks'] = description['max']
        rowDict['minObsClicks'] = description['min']
        rowDict['rangeObsClicks'] = description['max'] - description['min']
        rowList.append(rowDict)

    df = pd.DataFrame(rowList)
    data = pd.concat([data,df], axis=1)
    
    data.to_csv(outCSV, index=False)
    
#convert time stamp into timeDelta object for arithmetic operations
def parseTime(eventTime):
    timeString = eventTime.split()[1].split(':')
    t = datetime.timedelta(hours=int(timeString[0]), minutes=int(timeString[1]), seconds=int(timeString[2].split('.')[0]), milliseconds=int(timeString[2].split('.')[1]))
    return t


#truncate the training data by time into segments
#write truncated data to cutCSV
############################################
def timeCut(trainData, cutTime, cutCSV, outData=None):
    outData = pd.DataFrame(columns=['STUDENTID','Block','AccessionNumber','ItemType','Observable','ExtendedInfo','EventTime'])
    dataList = []
    curStudent = trainData.iloc[0,:]
    firstStudent = trainData.iloc[0,:]
    startTime = parseTime(firstStudent['EventTime'])
    endTime = parseTime(firstStudent['EventTime'])
    rowNum = 0
    for row in trainData.iterrows():
        rowNum += 1
        if ((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])):
            curStudent = row[1]
            startTime = parseTime(row[1].loc['EventTime'])
            endTime = startTime
        if (str(row[1].loc['EventTime']) != 'nan'):
            endTime = parseTime(row[1].loc['EventTime'])
            if((endTime-startTime).total_seconds()<cutTime):
                dataList.append(row[1])

    outData = pd.DataFrame(dataList,columns=['STUDENTID','Block','AccessionNumber','ItemType','Observable','ExtendedInfo','EventTime'])
    outData.to_csv(cutCSV,index=False)


#Used to split into True and False datasets for process mining using Fluxicon Disco
############################################
def tFSplit(trainData, labelData, trueCSV, falseCSV):
    curStudent = labelData.iloc[0, :]
    studentCount = 0
    falseList = []
    trueList = []
    rowCount = 0

    for row in trainData.iterrows():
        rowCount += 1
        if (rowCount % 100000 == 0):
            print(rowCount)
        # print(rowCount)
        if((row[1].loc['STUDENTID'] != curStudent.loc['STUDENTID'])) or row[0] == len(trainData)-1:
            studentCount += 1
            if (row[0] != len(trainData) - 1): curStudent = labelData.iloc[studentCount, :]

        curDict = row[1].to_dict()
        if(curStudent.loc['Target'] == True):
            curDict['Target'] = True
            trueList.append(curDict)
        else:
            curDict['Target'] = False
            falseList.append(curDict)

    trueData = pd.DataFrame(trueList)
    falseData = pd.DataFrame(falseList)
    trueData.to_csv(trueCSV, index=False)
    falseData.to_csv(falseCSV, index=False)

def concatAllScrapes(label, outCSV):
    path = '../TrainingData/'
    print(path+'scrape1_'+label)
    scrape1 = pd.read_csv(path+'scrape1_'+label+'.csv')
    scrape2 = pd.read_csv(path+'scrape2_'+label+'.csv')
    scrape3 = pd.read_csv(path+'scrape3_'+label+'.csv')
    scrape4 = pd.read_csv(path+'scrape4_'+label+'.csv')
    scrape5 = pd.read_csv(path+'scrape5_'+label+'.csv')
    scrape2 = scrape2.drop("StudentID", axis=1)
    scrape3 = scrape3.drop("StudentID", axis=1)
    scrape4 = scrape4.drop("StudentID", axis=1)
    scrape5 = scrape5.drop("StudentID", axis=1)
    allScrapes=pd.concat([scrape1,scrape2,scrape3,scrape4,scrape5],axis=1)
    allScrapes.to_csv(path+outCSV, index=False)
    

#Common processes - generated from Fluxicon Disco process maps
procs = ["Draw;Scratchwork Erase Mode On;Erase",
         "Scratchwork Mode Off;Vertical Item Scroll;Eliminate Choice",
         "Close Calculator;Open Calculator",
         "Lose Focus;Click Progress Navigator",
         "Receive Focus;Equation Editor Button",
         "Vertical Item Scroll;Lose Focus",
         "Vertical Item Scroll;Open Calculator;Click Choice",
         "Enter Item;Open Calculator",
         "Enter Item;Click Choice",
         "Enter Item;Receive Focus",
         "Enter Item;Eliminate Choice",
         "Enter Item;Next",
         "Enter Item;Vertical Item Scroll;Open Calculator",
         "Enter Item;Open Calculator;Calculator Buffer",
         "Enter Item;DropChoice",
         "Enter Item;Scratchwork Mode On",
         "Enter Item;Click Progress Navigator",
         "Enter Item;Increase Zoom",
         "Move Calculator;Exit Item",
         "Move Calculator;DropChoice",
         "Move Calculator;Receive Focus",
         "Move Calculator;Click Choice",
         "Open Calculator;Click Choice",
         "Close Calculator;Open Calculator",
         "Receive Focus;Lose Focus",
         "Vertical Item Scroll;Open Calculator",
         "Exit Item;Enter Item;Click Choice",
         "Exit Item;Enter Item",
         "Exit Item;Enter Item;Next",
         "Math Keypress;First Text Change",
         "Draw;Draw","Enter Item;Next;Exit Item",
        "Exit Item;Calculator Buffer;Close Calculator;Click Choice",
        "Close Calculator;Click Choice",
        "Exit Item;Calculator Buffer",
        "Enter Item;Open Calculator;Move Calculator;DropChoice",
        "Enter Item;Open Calculator;Move Calculator",
        "Click Choice;Next;Exit Item",
        "Click Choice;Next",
        "Enter Item;Eliminate Choice;Click Choice",
        "Vertical Item Scroll;Next",
        "Move Calculator;Eliminate Choice",
        "Click Choice;Click Progress Navigator",
        "Click Progress Navigator;Exit Item",
        "DropChoice;DropChoice",
        "Calculator Buffer;Enter Item",
        "Move Calculator;Move Calculator",
        "Clear Answer;Click Choice",
        "Exit Item;Calculator Buffer;Close Calculator",
        "Receive Focus;Math Keypress;Lose Focus",
        "Receive Focus;Math Keypress",
        "Draw;Clear Scratchwork"]


Open the training data and split it into 10, 20, and 30 minute data sets.

In [ ]:
with open('../TrainingData/data_a_train.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    timeCut(dataReader, 600, '../TrainingData/data_a_train_10.csv')
    timeCut(dataReader, 1200, '../TrainingData/data_a_train_20.csv')
    timeCut(dataReader, 1800, '../TrainingData/data_a_train_30.csv')

In [ ]:
with open('../TrainingData/data_a_train_10.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        labelReader = pd.read_csv(labelFile)
        scrape1(dataReader, '../TrainingData/scrape1_T10.csv', labelReader)
        
with open('../TrainingData/data_a_train_20.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        labelReader = pd.read_csv(labelFile)
        scrape1(dataReader, '../TrainingData/scrape1_T20.csv', labelReader)
        
with open('../TrainingData/data_a_train_30.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        labelReader = pd.read_csv(labelFile)
        scrape1(dataReader, '../TrainingData/scrape1_T30.csv', labelReader)
        
with open('../TrainingData/data_a_hidden_10.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        labelReader = pd.read_csv(labelFile)
        scrape1(dataReader, '../TrainingData/scrape1_H10.csv', labelReader, True)
        
with open('../TrainingData/data_a_hidden_20.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        labelReader = pd.read_csv(labelFile)
        scrape1(dataReader, '../TrainingData/scrape1_H20.csv', labelReader, True)
        
with open('../TrainingData/data_a_hidden_30.csv', newline='') as dataFile:
    with open('../TrainingData/data_train_label.csv', newline='') as labelFile:
        dataReader = pd.read_csv(dataFile)
        scrape1(dataReader, '../TrainingData/scrape1_H30.csv', labelReader, True)

In [ ]:
with open('../TrainingData/data_a_train_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_T10.csv')
        
with open('../TrainingData/data_a_train_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_T20.csv')
        
with open('../TrainingData/data_a_train_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_T30.csv')
        
with open('../TrainingData/data_a_hidden_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_H10.csv')
        
with open('../TrainingData/data_a_hidden_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_H20.csv')
        
with open('../TrainingData/data_a_hidden_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape2(dataReader,procs, '../TrainingData/scrape2_H30.csv')

In [ ]:
with open('../TrainingData/data_a_train_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_T10.csv')
        
with open('../TrainingData/data_a_train_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_T20.csv')
        
with open('../TrainingData/data_a_train_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_T30.csv')

with open('../TrainingData/data_a_hidden_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_H10.csv')
        
with open('../TrainingData/data_a_hidden_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_H20.csv')
        
with open('../TrainingData/data_a_hidden_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape3(dataReader, '../TrainingData/scrape3_H30.csv')

In [ ]:
with open('../TrainingData/data_a_train_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_T10.csv')
        
with open('../TrainingData/data_a_train_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_T20.csv')
        
with open('../TrainingData/data_a_train_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_T30.csv')

with open('../TrainingData/data_a_hidden_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_H10.csv')
        
with open('../TrainingData/data_a_hidden_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_H20.csv')
        
with open('../TrainingData/data_a_hidden_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape4(dataReader, '../TrainingData/scrape4_H30.csv')

In [ ]:
with open('../TrainingData/data_a_train_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_T10.csv')
        
with open('../TrainingData/data_a_train_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_T20.csv')
        
with open('../TrainingData/data_a_train_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_T30.csv')

with open('../TrainingData/data_a_hidden_10.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_H10.csv')
        
with open('../TrainingData/data_a_hidden_20.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_H20.csv')
        
with open('../TrainingData/data_a_hidden_30.csv', newline='') as dataFile:
    dataReader = pd.read_csv(dataFile)
    scrape5(dataReader, '../TrainingData/scrape5_H30.csv')

In [ ]:
concatAllScrapes('T10','FullScrapeT10.csv')
concatAllScrapes('T20','FullScrapeT20.csv')
concatAllScrapes('T30','FullScrapeT30.csv')
concatAllScrapes('H10','FullScrapeH10.csv')
concatAllScrapes('H20','FullScrapeH20.csv')
concatAllScrapes('H30','FullScrapeH30.csv')

In [ ]:
addStats('../TrainingData/FullScrapeT10.csv', '../TrainingData/FullScrapeT10.csv')
addStats('../TrainingData/FullScrapeT20.csv', '../TrainingData/FullScrapeT20.csv')
addStats('../TrainingData/FullScrapeT30.csv', '../TrainingData/FullScrapeT30.csv')
addStats('../TrainingData/FullScrapeH10.csv', '../TrainingData/FullScrapeH10.csv')
addStats('../TrainingData/FullScrapeH20.csv', '../TrainingData/FullScrapeH20.csv')
addStats('../TrainingData/FullScrapeH30.csv', '../TrainingData/FullScrapeH30.csv')